Please refer to Flipsidecrypto to collected data
https://flipsidecrypto.xyz/

Our code was forked from https://flipsidecrypto.xyz/siavashj/q/1sPLGdSteDlQ
and fixed already.

# Price

https://flipsidecrypto.xyz/edit/queries/4f0eddd4-78d3-4d70-a885-2cf05569e511

In [ ]:
 select
 date_trunc('day',hour) as daily,
 avg(price) as "Price"
from ethereum.price.ez_hourly_token_prices
where symbol ilike 'WETH'
group by 1
order by daily

# Transaction Flow

## DEX --> CEX

https://flipsidecrypto.xyz/edit/queries/33aeec6c-3aaf-4eff-bc32-7348ddd51acc

In [ ]:
WITH cexs AS (
    SELECT
        address AS cex_a,
        label AS cex
    FROM ethereum.core.dim_labels
    WHERE label_type = 'cex'
),
dexs AS (
    SELECT
        address AS dex_a,
        label AS dex
    FROM ethereum.core.dim_labels
    WHERE label_type = 'dex'
)
SELECT
    DATE_TRUNC('day', block_timestamp) AS date,
    COUNT(DISTINCT tx_hash) AS n_transfer,
    COUNT(origin_from_address) AS user,
    SUM(amount_usd) AS amount_usd,
    SUM(amount) AS amount_eth,
    cex,
    cex_a AS cex_address,
    dex
FROM
    ethereum.core.ez_token_transfers
JOIN cexs c ON to_address = cex_a
JOIN dexs ON from_address = dex_a
WHERE
    symbol = 'WETH'
    AND DATE_TRUNC('day', block_timestamp) >= '2021-01-01'
    AND DATE_TRUNC('day', block_timestamp) < '2023-07-01'
GROUP BY 1, 6, 7, 8
ORDER BY date;

## CEX --> DEX

https://flipsidecrypto.xyz/edit/queries/517725e2-a981-4d3e-8ae0-719d6eee2a8f

In [ ]:
 with cexs as (
select
 address as cex_a,
 label as cex
from ethereum.core.dim_labels
where label_type = 'cex'
),
dexs as (
select
 address as dex_a,
 label as dex
from ethereum.core.dim_labels
where label_type = 'dex'
)
select
 date_trunc('day',block_timestamp) as date,
 count(distinct tx_hash) as n_transfer,
 count(origin_from_address) as User,
 sum(amount_usd) as amount_usd,
 sum(amount) as amount_eth,
 cex,
 dex
from ethereum.core.ez_token_transfers join cexs c
 on from_address = cex_a
  join dexs on to_address = dex_a
WHERE symbol = 'WETH'
    AND DATE_TRUNC('day', block_timestamp) >= '2021-01-01'
    AND DATE_TRUNC('day', block_timestamp) < '2023-07-02'
group by 1,6,7
ORDER BY date;


## NetFlow

https://flipsidecrypto.xyz/edit/queries/de15f934-259c-46a2-8440-915d1726b412

In [ ]:
 with cexs as (
select
 address,
 label_type
from ethereum.core.dim_labels
where label_type = 'cex'
),
dexs as (
select
 address,
 label_type
from ethereum.core.dim_labels
where label_type = 'dex'
),
base as (
select
 block_timestamp,
 origin_from_address,
 from_address,
 to_address,
 amount,
 amount_usd
from ethereum.core.ez_token_transfers
where symbol = 'WETH'
AND DATE_TRUNC('day', block_timestamp) >= '2021-01-01'
    AND DATE_TRUNC('day', block_timestamp) < '2023-07-02'
),
to_dexs as (
select
 date_trunc('day',block_timestamp) as day,
 count(distinct origin_from_address) as user_to_dex,
 sum(amount) as eth_to_dex,
 sum(amount_usd) as usd_to_dex
from base join cexs
 on from_address = address
where to_address in (select address from dexs)
 group by 1
),
to_cexs as (
select
 date_trunc('day',block_timestamp) as date,
 count(distinct origin_from_address) as user_to_cex,
 sum(amount) as eth_to_cex,
 sum(amount_usd) as usd_to_cex
from base join dexs
 on from_address = address
where to_address in (select address from cexs)
 group by 1
)
select
 date,
 user_to_dex as "USER --> DEX",
 user_to_cex as "USER --> CEX",
 (user_to_cex - user_to_dex) as "Net User",
 eth_to_dex as "Amount --> DEX",
 eth_to_cex as "Amount --> CEX",
 (eth_to_cex - eth_to_dex) as "Net Amount ETH",
 usd_to_dex as "Amount $ --> DEX",
 usd_to_cex as "Amount $ --> CEX",
 (usd_to_cex - usd_to_dex) as "Net Amount $"
from to_dexs join to_cexs
 on date = day
ORDER BY date;